In [ ]:
%matplotlib inline

## Iris introduction course
# 6. Advanced Concepts

**Learning Outcome**: by the end of this section, you will be able to explain the capabilities and functionality of Iris Cubes and Coordinates.

**Duration:** 1.5 hour

**Overview:**<br>
6.1 [Load Callbacks](#callbacks)<br>
6.2 [Categorised Statistics](#categorical)<br>
6.3 [Performance tricks](#performance)<br>
6.4 [Final Exercise](#exercise)<br>
6.6 [Summary of the Section](#summary)

## Setup

In [ ]:
import iris
import numpy as np

In [ ]:
print(iris.__version__)
print(np.__version__)

## 6.1 Load callbacks<a id='callbacks'></a>

Sometimes important data exists in a filename rather than in the file itself, and it is desirable for it to become part of the cube's metadata.
For example, some early GloSea4 model runs recorded the "ensemble member number" (or "realization" in CF terms) in the filename, but not in actual PP metadata itself. As a result, loading the data yielded 2 cubes, rather than a single, fully merged, cube.

In [ ]:
fname = iris.sample_data_path('GloSea4', 'ensemble_00[34].pp')
for cube in iris.load(fname, 'surface_temperature'):
    print(cube, '\n', '--' * 50)

To resolve this we can define a function that gets called during the load process. This must take as arguments:

 * a cube,
 * a 2D field - either a PP field, a NetCDF variable or a GRIB message depending on the file format being loaded, and
 * a filename.

In this case, the function makes the necessary adjustments to include a "realization" coordinate. We pass this function to load, and the result is a successfully merged cube:

In [ ]:
import os
def realization_callback(cube, field, fname):
    basename = os.path.basename(fname)
    if not cube.coords('realization') and basename.startswith('ensemble_'):
        cube.add_aux_coord(iris.coords.DimCoord(np.int32(basename[-6:-3]),
                                                'realization'))

print(iris.load_cube(fname, callback=realization_callback))

## 6.2 Categorical coordinates for grouped statistics<a id='categorical'></a>

Sometimes we want to be able to categorise data before performing statistical operations on it. For example, we might want to categorise our data by "daylight maximum" or "seasonal mean" etc. Both of these categorisations would be based on the time coordinate.

The ``iris.coord_categorisation`` module provides convenience functions to add some common categorical coordinates, and provides a generalised function to allow each creation of custom categorisations. 

In [ ]:
import iris.coord_categorisation as coord_cat

filename = iris.sample_data_path('ostia_monthly.nc')
cube = iris.load_cube(filename, 'surface_temperature')
print(cube)

The cube loaded represents the monthly air_temperature from April 2006 through to October 2010. Let's add a categorisation coordinate to this cube to identify the climatological season (i.e "djf", "mam", "jja" or "son") of each time point:

In [ ]:
coord_cat.add_season(cube, 'time', name='clim_season')
print(cube)

In [ ]:
print(cube.coord('clim_season'))

We can now use the cube's ``aggregated_by`` method to "group by and aggregate" on the season, to produce the seasonal mean:

In [ ]:
seasonal_mean = cube.aggregated_by('clim_season', iris.analysis.MEAN)
print(seasonal_mean)

We can take this further by extracting by our newly created coordinate, producing a plot of the winter zonal mean:

In [ ]:
winter = seasonal_mean.extract(iris.Constraint(clim_season='djf'))

qplt.plot(winter.collapsed('latitude', iris.analysis.MEAN))
plt.title('Winter zonal mean surface temperature at $\pm5^{\circ}$ latitude')
plt.show()

#### Custom categorisation

Custom categorisation can be achieved with an arbitrary function. For example, the existing ``add_year`` categorisor takes the 'time' coordinate, and creates a 'year' coordinate. This could be achieved without using the available ``add_year`` by:

In [ ]:
def year_from_time(coord, point):
    return coord.units.num2date(point).year

coord_cat.add_categorised_coord(cube, 'year', cube.coord('time'),
                                year_from_time)

print(cube.coord('year'))

## 6.3 Performance tricks<a id='performance'></a>

This section details a few common tricks to improve the performance of your Iris code:

 * Data loading.
 * Load once, extract many times.

#### Make use of deferred loading of data

Sometimes it makes sense to load data before doing operations, other times it makes sense to do data reduction before loading:

In [ ]:
def zonal_sum(cube):
    """
    A really silly function to calculate the sum of the grid_longitude
    dimension.
    Don't use this in real life, instead consider doing:
    
        cube.collapsed('grid_longitude', iris.analysis.SUM)
    
    """
    total = 0
    for i, _ in enumerate(cube.coord('grid_longitude')):
        total += cube[..., i].data
    return total

In [ ]:
%%timeit
fname = iris.sample_data_path('uk_hires.pp')
pt = iris.load_cube(fname, 'air_potential_temperature')
result = zonal_sum(pt)

The exact same code, only with the data loaded upfront:

In [ ]:
%%timeit
fname = iris.sample_data_path('uk_hires.pp')
pt = iris.load_cube(fname, 'air_potential_temperature')
pt.data
result = zonal_sum(pt)

#### Load once, extract many times

Iris loading can be slow, particularly if the format stores 2d fields of a conceptually higher dimensional dataset, as is the case with GRIB and PP. To maximise load speed and avoid unncecessary processing, it is worth constraining the fields that are of interest *at load time*, but there is no caching, so loading a file twice will be twice as slow.

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
model_levels = [1, 4, 7, 16]

In [ ]:
%%timeit
for model_level in model_levels:
    pt = iris.load_cube(fname,
                        iris.Constraint('air_potential_temperature',
                                        model_level_number=model_level))

In [ ]:
%%timeit
cubes = iris.load(fname)
for model_level in model_levels:
    pt = cubes.extract(iris.Constraint('air_potential_temperature',
                                       model_level_number=model_level),
                       strict=True)

For files with lots of different phenomenon this can be improved further by loading only the phenomenon (and in this case just the model levels of interest):

In [ ]:
%%timeit
cube = iris.load(fname,
                 iris.Constraint('air_potential_temperature',
                                 model_level_number=model_levels))
for model_level in model_levels:
    pt = cube.extract(iris.Constraint(model_level_number=model_level))

## 6.4 Final Exercise<a id='exercise'></a>

Produce a set of plots that provide a comparison of decadal-mean air temperatures over North America:

**Part 1**

Load 'A1B_north_america.nc' from the Iris sample data.

**Part 2**

Extract just data from the year 1980 and beyond from the loaded data.

**Part 3**

Define a function that takes a coordinate and a single time point as arguments, and returns the decade. For example, your function should return 2010 for the following:

```python
time = iris.coords.DimCoord([10], 'time',
                           units='days since 2018-01-01')
print(your_decade_function(time, time.points[0]))
```

**Part 4**

Add a "decade" coordinate to the loaded cube using your function and the coord categorisation module.

**Part 5**

Calculate the decadal means cube for this scenario.

**Part 6**

Create a figure with 3 rows and 4 columns displaying the decadal means, with the decade displayed prominently in each axes' title.

## 5.4 Summary of Section: Advanced techniques<a id='summary'></a>

In this section we learnt:
* load callbacks can be used to capture additional control metadata during loading
* special facilities are provided for performing categorical statistics
* lazy loading can be used to enhance code performance
